This script is to test the 'recognition' module with a bit large data about running time and accuracy.

1) divide the test data into two parts (done separately)
    - images to be enrolled
    - images to test
2) tracking the time spent on each search
3) calculate the accuracy: # of correctly recognized image / # of total test images

In [95]:
# settings
host = 'http://172.29.34.77/'
api_key = 'f8b19cb08f93a891e33bda8e682a83b8cf7a3613' # expire in one day
appID = '4178165796'
headers = {'Authorization': 'Token ' + api_key}

enroll_root = './compact_vgg_faces_nogif/enroll'
test_root = './compact_vgg_faces_nogif/test'

In [96]:
import requests, os

def add_person(name):
    url = os.path.join(host, 'persons/')
    
    data = {'appID': appID, 'name':name}
    
    response = requests.post(url, data=data, headers=headers)
    
    if not str(response.status_code).startswith('2'):
        print('Add person failed!<%s>'%(response.status_code))
        return
    
    return response.json()['userID']

In [97]:
import requests, os, tempfile

def enroll_face(userID, facepath):
    
    url = os.path.join(host, 'faces/')
    
    files = {'image': open(facepath, 'rb')}
    data = {'appID': appID, 'userID': userID}
    
    response = requests.post(url, files=files, data=data, headers=headers)
    
    if not str(response.status_code).startswith('2'):
        print('Enroll_face failed!<%s>'%(response.status_code))
        return
    
    return response.status_code == 201

In [117]:
import requests, os, tempfile

def recognize(facepath, claimed_userID, feature, classifier):
    
    url = os.path.join(host, 'commands/recognition/')
    
    files = {'face': open(facepath, 'rb')}
    data = {'appID': appID, 'classifier': classifier}
    if feature is not None:
        data['feature'] = feature
    
    response = requests.post(url, headers=headers, files=files, data=data, timeout=None)
    
    if not str(response.status_code).startswith('2'):
        print('Recognize failed!<%s>'%(response.status_code))
        return None
    
    #print(response.json()['userID'],claimed_userID)
    
    return response.json()['userID'] == claimed_userID

Enroll faces

In [99]:
import os, sys
persons = {}
max_per_person = 150
for person in os.listdir(enroll_root): 
    userID = add_person(person)
    persons[person] = userID

    count = 0
    for face_filename in os.listdir(os.path.join(enroll_root, person)):
        path = os.path.join(enroll_root, person, face_filename)
        enroll_face(userID, path)
        count += 1
        if count > max_per_person:
            break;
    print("Person %s has %d faces enrolled."%(person, count))
print(persons)

Person 43 has 151 faces enrolled.
Enroll_face failed!<500>
Person 50 has 151 faces enrolled.
Enroll_face failed!<400>
Person 28 has 151 faces enrolled.
Person 54 has 151 faces enrolled.
Person 40 has 151 faces enrolled.
Person 30 has 151 faces enrolled.
Person 3 has 151 faces enrolled.
Person 34 has 151 faces enrolled.
Person 15 has 151 faces enrolled.
Person 55 has 112 faces enrolled.
Person 48 has 151 faces enrolled.
Person 32 has 151 faces enrolled.
Person 21 has 151 faces enrolled.
Person 46 has 151 faces enrolled.
Person 16 has 151 faces enrolled.
Person 27 has 151 faces enrolled.
Person 52 has 151 faces enrolled.
Person 44 has 151 faces enrolled.
Person 18 has 151 faces enrolled.
Person 41 has 151 faces enrolled.
Person 36 has 151 faces enrolled.
Person 37 has 151 faces enrolled.
Person 11 has 151 faces enrolled.
Person 23 has 151 faces enrolled.
Person 29 has 151 faces enrolled.
Person 5 has 151 faces enrolled.
Person 12 has 151 faces enrolled.
Person 20 has 151 faces enrolled.


In [42]:
# get all person info 
url = os.path.join(host, 'persons/')
data = {'appID': appID}
response = requests.get(url, headers=headers, data=data)
persons = {}
for person in response.json():
    persons[person['name']] = person['userID']
print(persons)

['http://172.29.34.77/faces/1/', 'http://172.29.34.77/faces/2/', 'http://172.29.34.77/faces/3/', 'http://172.29.34.77/faces/4/', 'http://172.29.34.77/faces/5/', 'http://172.29.34.77/faces/6/', 'http://172.29.34.77/faces/7/', 'http://172.29.34.77/faces/8/', 'http://172.29.34.77/faces/9/', 'http://172.29.34.77/faces/10/', 'http://172.29.34.77/faces/11/', 'http://172.29.34.77/faces/12/', 'http://172.29.34.77/faces/13/', 'http://172.29.34.77/faces/14/', 'http://172.29.34.77/faces/15/', 'http://172.29.34.77/faces/16/', 'http://172.29.34.77/faces/17/', 'http://172.29.34.77/faces/18/', 'http://172.29.34.77/faces/19/', 'http://172.29.34.77/faces/20/', 'http://172.29.34.77/faces/21/', 'http://172.29.34.77/faces/22/', 'http://172.29.34.77/faces/23/', 'http://172.29.34.77/faces/24/', 'http://172.29.34.77/faces/25/', 'http://172.29.34.77/faces/26/', 'http://172.29.34.77/faces/27/', 'http://172.29.34.77/faces/28/', 'http://172.29.34.77/faces/29/', 'http://172.29.34.77/faces/30/', 'http://172.29.34.

In [126]:
def evaluation(feature, classifier, worker_name):
    start = datetime.now()
    total = 0
    hits = 0
    for person in os.listdir(test_root):
        user_id = persons[person]
        for facename in os.listdir(os.path.join(test_root, person)):
            face_path = os.path.join(test_root, person, facename)
            is_hit = recognize(face_path, user_id, feature, classifier)
            if is_hit == None:
                continue
            total += 1
            hits += is_hit
            
            if total % 50 == 0:
                print('[%s]done: %d'%(worker_name, total))
    print("[%s]Accuracy: %f, time: %s"%(worker_name, hits/total, datetime.now()-start))

In [127]:
def work1():
    evaluation('pca', 'nearest_neighbor', 'PCA_NN')
    
def work2():
    evaluation('pca', 'svm', 'PCA_SVM')
    
def work3():
    evaluation('lda', 'nearest_neighbor', 'LDA_NN')
    
def work4():
    evaluation('lda', 'svm', 'LDA_SVM')
    
def work5():
    evaluation('lbp', 'nearest_neighbor', 'LBP_NN')
    
def work6():
    evaluation('lbp', 'svm', 'lBP_SVM')
    
def work7():
    evaluation('hog', 'nearest_neighbor', 'HOG_NN')

def work8():
    evaluation('hog', 'svm', 'HOG_SVM')

def work9():
    evaluation(None, 'nearest_neighbor', 'openface_NN')
    
def work10():
    evaluation(None, 'svm', 'openface_SVM')
    
from multiprocessing import Process

proc = []
proc.append(Process(target=work1))
proc.append(Process(target=work2))
proc.append(Process(target=work3))
proc.append(Process(target=work4))
proc.append(Process(target=work5))
proc.append(Process(target=work6))
proc.append(Process(target=work7))
proc.append(Process(target=work8))
#proc.append(Process(target=work9))

for p in proc:
    p.start()
    
for p in proc:
    p.join()


[HOG_SVM]done: 50
[lBP_SVM]done: 50
[PCA_NN]done: 50
[HOG_NN]done: 50
[LDA_NN]done: 50
[LBP_NN]done: 50
[HOG_SVM]done: 100
[lBP_SVM]done: 100
[PCA_NN]done: 100
[HOG_NN]done: 100
[LDA_NN]done: 100
[HOG_SVM]done: 150
[LBP_NN]done: 100
[lBP_SVM]done: 150
[PCA_NN]done: 150
[HOG_NN]done: 150
[HOG_SVM]done: 200
[LDA_NN]done: 150
[LBP_NN]done: 150
[lBP_SVM]done: 200
[HOG_SVM]done: 250
[PCA_NN]done: 200
[HOG_NN]done: 200
[LDA_NN]done: 200
[HOG_SVM]done: 300
[lBP_SVM]done: 250
[LBP_NN]done: 200
[PCA_NN]done: 250
[HOG_NN]done: 250
[HOG_SVM]done: 350
[LDA_NN]done: 250
[lBP_SVM]done: 300
[PCA_NN]done: 300
[LBP_NN]done: 250
[HOG_NN]done: 300
[HOG_SVM]done: 400
[lBP_SVM]done: 350
[LDA_NN]done: 300
[PCA_NN]done: 350
[HOG_SVM]done: 450
[HOG_NN]done: 350
[LBP_NN]done: 300
[lBP_SVM]done: 400
[HOG_SVM]done: 500
[LDA_NN]done: 350
[PCA_NN]done: 400
[HOG_NN]done: 400
[LBP_NN]done: 350
[lBP_SVM]done: 450
[HOG_SVM]done: 550
[PCA_NN]done: 450
[LDA_NN]done: 400
[HOG_NN]done: 450
[lBP_SVM]done: 500
[HOG_SVM]done

Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize f

Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize f

Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize f

Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize f

...

Process Process-135:
Traceback (most recent call last):
  File "/home/xhoutao/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/xhoutao/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-127-ced5a3b468d8>", line 5, in work2
    evaluation('pca', 'svm', 'PCA_SVM')
  File "<ipython-input-126-6edc8a9c6b1e>", line 17, in evaluation
    print("[%s]Accuracy: %f, time: %s"%(worker_name, hits/total, datetime.now()-start))


Recognize failed!<502>


ZeroDivisionError: division by zero


Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>


Process Process-137:
Traceback (most recent call last):
  File "/home/xhoutao/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/xhoutao/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-127-ced5a3b468d8>", line 11, in work4
    evaluation('lda', 'svm', 'LDA_SVM')
  File "<ipython-input-126-6edc8a9c6b1e>", line 17, in evaluation
    print("[%s]Accuracy: %f, time: %s"%(worker_name, hits/total, datetime.now()-start))
ZeroDivisionError: division by zero


In [60]:
from datetime import datetime
start = datetime.now()
recognize('/home/xhoutao/Desktop/face/RESTful_Face_Web/test/compact_vgg_faces/test/1/1.jpg', '0760397714', 'hog', 'svm')
print(datetime.now()-start)

0:04:24.590710


In [78]:
from datetime import datetime
print(datetime.now())

2017-08-26 12:49:42.022606


In [105]:
from datetime import datetime
start = datetime.now()
work9()  # openface + nearest neighbor
print('classify time: ', datetime.now() - start)

[openface_NN]done: 50
[openface_NN]done: 100
[openface_NN]done: 150
[openface_NN]done: 200
[openface_NN]done: 250
[openface_NN]done: 300
[openface_NN]done: 350
[openface_NN]done: 400
[openface_NN]done: 450
[openface_NN]done: 500
[openface_NN]done: 550
[openface_NN]done: 600
[openface_NN]done: 650
[openface_NN]done: 700
[openface_NN]done: 750
[openface_NN]done: 800
[openface_NN]done: 850
[openface_NN]done: 900
[openface_NN]done: 950
[openface_NN]done: 1000
[openface_NN]done: 1050
[openface_NN]Accuracy: 0.640221
classify time:  0:05:10.913723


In [120]:
from datetime import datetime
start = datetime.now()
work10() # openface + svm (RBF) regularization = 1
print('classify time: ', datetime.now() - start)

[openface_SVM]done: 50
[openface_SVM]done: 100
[openface_SVM]done: 150
[openface_SVM]done: 200
[openface_SVM]done: 250
[openface_SVM]done: 300
[openface_SVM]done: 350
[openface_SVM]done: 400
[openface_SVM]done: 450
[openface_SVM]done: 500
[openface_SVM]done: 550
[openface_SVM]done: 600
[openface_SVM]done: 650
[openface_SVM]done: 700
[openface_SVM]done: 750
[openface_SVM]done: 800
[openface_SVM]done: 850
[openface_SVM]done: 900
[openface_SVM]done: 950
[openface_SVM]done: 1000
[openface_SVM]done: 1050
[openface_SVM]Accuracy: 0.546125


In [121]:
from datetime import datetime
start = datetime.now()
work10() # openface + linear svm regularization = 1
print('classify time: ', datetime.now() - start)

[openface_SVM]done: 50
[openface_SVM]done: 100
[openface_SVM]done: 150
[openface_SVM]done: 200
[openface_SVM]done: 250
[openface_SVM]done: 300
[openface_SVM]done: 350
[openface_SVM]done: 400
[openface_SVM]done: 450
[openface_SVM]done: 500
[openface_SVM]done: 550
[openface_SVM]done: 600
[openface_SVM]done: 650
[openface_SVM]done: 700
[openface_SVM]done: 750
[openface_SVM]done: 800
[openface_SVM]done: 850
[openface_SVM]done: 900
[openface_SVM]done: 950
[openface_SVM]done: 1000
[openface_SVM]done: 1050
[openface_SVM]Accuracy: 0.725092


In [132]:
proc = []
proc.append(Process(target=work2))
#proc.append(Process(target=work4))

for p in proc:
    p.start()
    
for p in proc:
    p.join()

Recognize failed!<504>
Recognize failed!<504>


Process Process-150:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/xhoutao/anaconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py", line 379, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

  File "/home/xhoutao/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/xhoutao/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-127-ced5a3b468d8>", line 5, in work2
    evaluation('pca', 'svm', 'PCA_SVM')
  File "<ipython-input-126-6edc8a9c6b1e>", line 9, in evaluation
    is_hit = recognize(face_path, user_id, feature, classifier)
  File "<ipython-input-117-8cd87600b7b1>", line 12, in recognize
    response = requests.post(url,

KeyboardInterrupt: 

  File "/home/xhoutao/anaconda3/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/home/xhoutao/anaconda3/lib/python3.5/socket.py", line 576, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


In [133]:
work6()

Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize failed!<502>
Recognize f

KeyboardInterrupt: 